In [1]:
import numpy as np

In [2]:
sample = np.arange(6*40).reshape(-1, 6)
label = -np.arange(2*40*3, step=3).reshape(-1, 2)

print(sample)

[[  0   1   2   3   4   5]
 [  6   7   8   9  10  11]
 [ 12  13  14  15  16  17]
 [ 18  19  20  21  22  23]
 [ 24  25  26  27  28  29]
 [ 30  31  32  33  34  35]
 [ 36  37  38  39  40  41]
 [ 42  43  44  45  46  47]
 [ 48  49  50  51  52  53]
 [ 54  55  56  57  58  59]
 [ 60  61  62  63  64  65]
 [ 66  67  68  69  70  71]
 [ 72  73  74  75  76  77]
 [ 78  79  80  81  82  83]
 [ 84  85  86  87  88  89]
 [ 90  91  92  93  94  95]
 [ 96  97  98  99 100 101]
 [102 103 104 105 106 107]
 [108 109 110 111 112 113]
 [114 115 116 117 118 119]
 [120 121 122 123 124 125]
 [126 127 128 129 130 131]
 [132 133 134 135 136 137]
 [138 139 140 141 142 143]
 [144 145 146 147 148 149]
 [150 151 152 153 154 155]
 [156 157 158 159 160 161]
 [162 163 164 165 166 167]
 [168 169 170 171 172 173]
 [174 175 176 177 178 179]
 [180 181 182 183 184 185]
 [186 187 188 189 190 191]
 [192 193 194 195 196 197]
 [198 199 200 201 202 203]
 [204 205 206 207 208 209]
 [210 211 212 213 214 215]
 [216 217 218 219 220 221]
 

In [16]:
#print(label)

In [4]:
width = 2

start = sample[0:-width*2+1]
second = sample[1:-width*2+2]

stacked = np.stack([sample[i:-width*2+1+i if -width*2+1+i != 0 else None] for i in range(width*2)], 1)
relevant_action = label[width-1:-width]

print(stacked)

[[[  0   1   2   3   4   5]
  [  6   7   8   9  10  11]
  [ 12  13  14  15  16  17]
  [ 18  19  20  21  22  23]]

 [[  6   7   8   9  10  11]
  [ 12  13  14  15  16  17]
  [ 18  19  20  21  22  23]
  [ 24  25  26  27  28  29]]

 [[ 12  13  14  15  16  17]
  [ 18  19  20  21  22  23]
  [ 24  25  26  27  28  29]
  [ 30  31  32  33  34  35]]

 [[ 18  19  20  21  22  23]
  [ 24  25  26  27  28  29]
  [ 30  31  32  33  34  35]
  [ 36  37  38  39  40  41]]

 [[ 24  25  26  27  28  29]
  [ 30  31  32  33  34  35]
  [ 36  37  38  39  40  41]
  [ 42  43  44  45  46  47]]

 [[ 30  31  32  33  34  35]
  [ 36  37  38  39  40  41]
  [ 42  43  44  45  46  47]
  [ 48  49  50  51  52  53]]

 [[ 36  37  38  39  40  41]
  [ 42  43  44  45  46  47]
  [ 48  49  50  51  52  53]
  [ 54  55  56  57  58  59]]

 [[ 42  43  44  45  46  47]
  [ 48  49  50  51  52  53]
  [ 54  55  56  57  58  59]
  [ 60  61  62  63  64  65]]

 [[ 48  49  50  51  52  53]
  [ 54  55  56  57  58  59]
  [ 60  61  62  63  64  65]
  [ 

In [4]:
ep_obs = [[]]*2

print(ep_obs)

ep_obs[0].append(1)

print(ep_obs)

[[], []]
[[1], [1]]


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import numpy as np
import wandb
import os
import random
from time import time

from lib.data import EpisodeDataset
from lib.neural_networks import FullyConnected, ConvNet

In [2]:
TRAIN_PATH = '/home/bdemoss/Desktop/RL IDM/Episode Data/Random Data/30 TPS/3v3 Test'

train_dataset = EpisodeDataset(TRAIN_PATH, width=10, include_change=False)

Loading Episode Data
Finished Loading Episode Data


In [3]:
Train_Data_Loader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=16)

In [4]:
for obs, label, add_data in Train_Data_Loader:
    obs = obs[0]
    label = label[0]
    add_data = add_data[0]

    print(obs[45, 0:2, 0:21])
    print(label[45])
    break

tensor([[-0.6395,  0.2090,  0.5945, -0.8108,  0.1432,  0.5676,  0.4745, -0.4070,
          0.7805,  0.6147,  0.1433, -0.2163,  0.2903, -1.2888, -0.4619,  0.8292,
          0.0000, -0.6006, -0.2833,  0.0645, -0.6820],
        [-0.6193,  0.2138,  0.5873, -0.8781,  0.1647,  0.4492,  0.3491, -0.4215,
          0.8370,  0.6019,  0.1456, -0.2176, -0.1194, -1.3741, -0.1662,  0.8181,
          0.0000, -0.6023, -0.2587, -0.0334, -0.1962]])
tensor([1., 2., 0., 2., 1., 0., 0., 0.], dtype=torch.float64)


In [6]:
from lib.utils import quats_to_rot_mtx

In [9]:
test = np.array([[0.1300, 0.2788, -0.1790, 0.9345], [0.1300, 0.2788, -0.1790, 0.9345]])

print(quats_to_rot_mtx(test))

[[[-0.81073368 -0.34279388  0.47455585]
  [ 0.14316523 -0.90211415 -0.407055  ]
  [ 0.56763951 -0.2620733   0.78045049]]

 [[-0.81073368 -0.34279388  0.47455585]
  [ 0.14316523 -0.90211415 -0.407055  ]
  [ 0.56763951 -0.2620733   0.78045049]]]
